In [ ]:
pip install folium

# Library Load

In [15]:
import folium
import pandas as pd
import numpy as np 
import sklearn
import geopy
from sklearn.cluster import DBSCAN
from geopy.distance import great_circle
from sklearn.neighbors import BallTree

In [17]:
# 라이브러리 버전 출력
print(f"folium version: {folium.__version__}")
print(f"pandas version: {pd.__version__}")
print(f"numpy version: {np.__version__}")
print(f"scikit-learn version(DBSCAN, BallTree): {sklearn.__version__}")
print(f"geopy version(great_circle): {geopy.__version__}")

folium version: 0.19.4
pandas version: 2.1.4
numpy version: 1.26.4
scikit-learn version(DBSCAN, BallTree): 1.4.2
geopy version(great_circle): 2.4.1


# Data Load

In [1]:
d4 = pd.read_csv('4.성남시_표제부.csv')
d7 = pd.read_csv('7.성남시_지식산업센터.csv')

d7['vacancy_rate_2101'] = 1 - d7['cpn_in_2101']/d7['tot_cpn']
d7['vacancy_rate_2201'] = 1 - d7['cpn_in_2201']/d7['tot_cpn']
d7['vacancy_rate_2302'] = 1 - d7['cpn_in_2302']/d7['tot_cpn']
d7['vacancy_rate_2402'] = 1 - d7['cpn_in_2402']/d7['tot_cpn']
d7['vacancy_rate_2406'] = 1 - d7['cpn_in_2406']/d7['tot_cpn']

C:\Users\황회선\AppData\Local\Temp\ipykernel_31856\551751979.py:3: DtypeWarning: Columns (14,15,34,58,59,60,61,67) have mixed types. Specify dtype option on import or set low_memory=False.
  d4 = pd.read_csv('4.성남시_표제부.csv')


# Main

In [ ]:
# d4에 신주소를 기반으로 위도,경도를 찾아서 매칭해주는코드 (안돌려도 됩니다)

# pip install geopy

# from geopy.geocoders import Nominatim
# from geopy.extra.rate_limiter import RateLimiter

# # Geolocator 설정 (Nominatim 사용)
# geolocator = Nominatim(user_agent="geoapi")
# geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)  # API 호출 속도 조절

# # 함수 정의
# def get_lat_lon(address):
#     try:
#         location = geocode(address)
#         if location:
#             return location.latitude, location.longitude
#         else:
#             return None, None
#     except Exception as e:
#         print(f"Error: {e}, Address: {address}")
#         return None, None

# # 위도와 경도 컬럼 추가
# d4[['lat', 'lon']] = d4['new_plat_plc'].apply(lambda x: pd.Series(get_lat_lon(x)))

# d4[['lat', 'lon']]

In [ ]:
# ✅ Haversine 거리 기반 DBSCAN 클러스터링 함수
def cluster_d7(df, eps_km=1.0, min_samples=2):
    """
    d7(지산센)의 위치를 밀도 기반으로 클러스터링하여 대표 지점을 선정하는 함수

    :param df: 지산센 DataFrame (lat, lon 필수)
    :param eps_km: 클러스터 반경 (km)
    :param min_samples: 최소 클러스터 포함 개수
    :return: 클러스터링된 d7 (대표 lat, lon 포함)
    """
    # 위경도를 라디안 변환
    coords = np.radians(df[['lat', 'lon']].values)

    # DBSCAN 실행 (Haversine 거리 사용)
    kms_per_radian = 6371.0088  # 지구 반지름 (km)
    db = DBSCAN(eps=eps_km / kms_per_radian, min_samples=min_samples, metric='haversine')
    df['cluster'] = db.fit_predict(coords)

    # 클러스터 -1은 노이즈 데이터이므로 제외
    clustered_df = df[df['cluster'] != -1]

    # ✅ 각 클러스터별 대표 중심점(lat, lon) 계산
    cluster_centers = clustered_df.groupby('cluster')[['lat', 'lon']].mean().reset_index()
    # 클러스터 번호를 기반으로 region_info 생성
    cluster_centers['region_info'] = 'Cluster ' + (cluster_centers['cluster'] + 1).astype(str)

    return cluster_centers

# ✅ d7 클러스터링 실행
d7_clustered = cluster_d7(d7, eps_km=1.0, min_samples=2)

# 결과 확인
print(d7_clustered)

In [ ]:
# 위도, 경도를 라디안으로 변환하는 함수 (NaN 처리 추가)
def to_radians(df):
    df = df.copy()
    df['lat_r'] = np.radians(df['lat'])
    df['lon_r'] = np.radians(df['lon'])
    return df

# Haversine 거리 기반 최근접 탐색 함수 (NaN 고려)
def match_nearest(df1, df2):
    # NaN이 있는 경우, 그대로 유지하도록 변환
    df1 = to_radians(df1)
    df2 = to_radians(df2)

    # NaN이 있는 경우, 처리할 수 있도록 마스크 생성
    valid_mask = df1[['lat_r', 'lon_r']].notna().all(axis=1)

    # BallTree 생성 (haversine 거리 사용, NaN 제외)
    tree = BallTree(df2[['lat_r', 'lon_r']].dropna().values, metric='haversine')

    # 최근접 이웃 찾기 (NaN이 있는 행은 제외)
    distances = np.full(len(df1), np.nan)
    indices = np.full(len(df1), np.nan)

    if valid_mask.any():
        valid_distances, valid_indices = tree.query(df1.loc[valid_mask, ['lat_r', 'lon_r']].values, k=1)
        distances[valid_mask] = valid_distances.flatten()
        indices[valid_mask] = valid_indices.flatten()

    # 거리(km 단위 변환: 지구 반지름 6371km)
    df1['matched_lat'] = np.nan
    df1['matched_lon'] = np.nan
    df1['matched_region'] = np.nan
    df1['distance_km'] = distances * 6371  # 거리 (km)

    # 유효한 값이 있는 경우만 매칭
    valid_indices = indices[valid_mask].astype(int)
    df1.loc[valid_mask, 'matched_lat'] = df2.iloc[valid_indices]['lat'].values
    df1.loc[valid_mask, 'matched_lon'] = df2.iloc[valid_indices]['lon'].values
    df1.loc[valid_mask, 'matched_region'] = df2.iloc[valid_indices]['region_info'].values # 지산센이름

    return df1.drop(columns=['lat_r', 'lon_r'])  # 라디안 컬럼 삭제 후 반환

# 예제 데이터프레임 생성

# 최근접 매칭 실행
data_matched = match_nearest(d4, d7_clustered)

# 결과 출력
print(data_matched.head())

In [ ]:
# NaN 값 처리 함수 (NaN 값이 있는 행 제거)
def remove_na(df):
    return df.dropna(subset=['lat', 'lon'])

# 1. d4 건물 정보에 매칭된 d7 클러스터 정보 추가하기
def match_d4_to_d7_clusters(d4, d7_clustered):
    # NaN 값을 제거
    d4_cleaned = remove_na(d4)
    d7_clustered_cleaned = remove_na(d7_clustered)

    print(f"Cleaned d4 size: {d4_cleaned.shape}")
    print(f"Cleaned d7_clustered size: {d7_clustered_cleaned.shape}")

    # BallTree를 사용해서 d4의 건물 위치와 d7 클러스터 중심점과의 거리를 계산
    tree = BallTree(np.radians(d7_clustered_cleaned[['lat', 'lon']].values), metric='haversine')
    distances, indices = tree.query(np.radians(d4_cleaned[['lat', 'lon']].values), k=1)

    # 각 건물에 대해 매칭된 d7 클러스터의 region_info 값을 추가
    d4_cleaned['matched_region_info'] = d7_clustered_cleaned.iloc[indices.flatten()]['region_info'].values

    return d4_cleaned

# d4 건물에 매칭된 클러스터 정보 추가
d4_matched = match_d4_to_d7_clusters(d4, d7_clustered)

# 2. folium으로 시각화할 때, 매칭된 클러스터 번호에 따라 색상 설정
def create_map(d7_clustered, d4_matched):
    m = folium.Map(location=[37.5665, 126.978], zoom_start=10)

    # d7 클러스터의 대표 지산센을 큰 네모로 표시
    for idx, row in d7_clustered.iterrows():
        folium.Marker(
            location=[row['lat'], row['lon']],
            popup=row['region_info'],
            icon=folium.Icon(color='blue')
        ).add_to(m)

    # d4 건물들을 매칭된 지산센 클러스터 색깔에 맞게 점으로 표시
    for idx, row in d4_matched.iterrows():
        region_info = row['matched_region_info']
        
        # region_info에 맞는 색상을 설정 (예: Cluster 1 -> red, Cluster 2 -> green, ... 또는 다른 방식으로 처리)
        if region_info == 'Cluster 1':
            color = 'red'
        elif region_info == 'Cluster 2':
            color = 'green'
        elif region_info == 'Cluster 3':
            color = 'blue'
        else:
            color = 'gray'  # 기본 색

        folium.CircleMarker(
            location=[row['lat'], row['lon']],
            radius=5,
            color=color,
            fill=True,
            fill_color=color,
            fill_opacity=0.6
        ).add_to(m)

    return m

# d7 클러스터와 d4 매칭된 건물 정보로 지도 생성
m = create_map(d7_clustered, d4_matched)
m.save('center_clusters_with_buildings_map.html')  # 지도 저장
m

In [ ]:
d4_matched.groupby(['matched_region_info']).count().iloc[:,0].sort_values(ascending=False)

In [ ]:
# 클러스터별 공실률 경향 (2클러스터를 제외하곤 다 수가 1~2개여서 신뢰도는 많이 없음.)
d7.groupby(['cluster'])[d7.columns[16:21]].agg(['min','mean','median','max','count']).T